# Lab 1.7.5: MNIST Example

**Module:** 1.7 - Domain 1 Capstone: MicroGrad+  
**Time:** 1.5 hours  
**Difficulty:** ⭐⭐⭐⭐

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- [ ] Train a neural network on real handwritten digit images
- [ ] Achieve >95% accuracy using your MicroGrad+ library
- [ ] Visualize the learning process and predictions
- [ ] Understand how your library performs on a real problem

---

## 📚 Prerequisites

- Completed: Labs 1.7.1-1.7.4
- Knowledge of: Image classification basics

---

## 🌍 Real-World Context

MNIST (Modified National Institute of Standards and Technology) is the "Hello World" of machine learning. It contains 70,000 grayscale images of handwritten digits (0-9), each 28x28 pixels.

Despite being simple, MNIST helped establish key ML practices:
- Benchmarking model performance
- Developing training techniques
- Testing new architectures

---

## 🧒 ELI5: Image Classification

> **Imagine teaching a child to recognize numbers.**
>
> You show them many examples: "This is a 3. This is also a 3. This different-looking one is also a 3."
>
> Over time, they learn patterns:
> - "3's have two bumps on the right"
> - "8's look like two circles stacked"
> - "1's are just a vertical line"
>
> **In AI terms:**
> - Each image is 784 numbers (28x28 pixels, 0-255 grayscale)
> - The neural network learns which pixel patterns indicate which digit
> - After training, it can recognize digits it's never seen before!

In [ ]:
# Setup
import numpy as np
import matplotlib.pyplot as plt
import sys
import time
import gzip
import os
import socket
from urllib import request
from pathlib import Path

# Robust path resolution - works regardless of working directory
def _find_module_root():
    """Find the module root directory containing micrograd_plus."""
    current = Path.cwd()
    for parent in [current] + list(current.parents):
        if (parent / 'micrograd_plus' / '__init__.py').exists():
            return str(parent)
    return str(Path.cwd().parent)

sys.path.insert(0, _find_module_root())

from micrograd_plus import (
    Tensor, Linear, ReLU, Sigmoid, Dropout,
    CrossEntropyLoss, Adam, SGD, Sequential
)
from micrograd_plus.utils import set_seed, DataLoader

set_seed(42)

print("Setup complete!")

---

## Part 1: Loading MNIST Data

In [ ]:
def download_mnist(path=None, timeout=30):
    """
    Download MNIST dataset with fallback URLs and timeout.
    
    Uses multiple mirror URLs to ensure reliability:
    1. PyTorch/OSSCI S3 mirror (most reliable)
    2. Original yann.lecun.com (backup)
    
    Args:
        path: Directory to save files.
        timeout: Download timeout in seconds.
    """
    # Resolve path relative to module root if not specified
    if path is None:
        path = os.path.join(_find_module_root(), 'data')
    os.makedirs(path, exist_ok=True)
    
    # Primary and fallback URLs
    urls = [
        'https://ossci-datasets.s3.amazonaws.com/mnist/',  # PyTorch mirror (reliable)
        'http://yann.lecun.com/exdb/mnist/',  # Original (sometimes unreliable)
    ]
    
    files = [
        'train-images-idx3-ubyte.gz',
        'train-labels-idx1-ubyte.gz',
        't10k-images-idx3-ubyte.gz',
        't10k-labels-idx1-ubyte.gz'
    ]
    
    for f in files:
        filepath = os.path.join(path, f)
        if not os.path.exists(filepath):
            downloaded = False
            for base_url in urls:
                try:
                    print(f"Downloading {f} from {base_url}...")
                    # Set timeout using socket
                    old_timeout = socket.getdefaulttimeout()
                    socket.setdefaulttimeout(timeout)
                    try:
                        request.urlretrieve(base_url + f, filepath)
                        downloaded = True
                        print(f"  Success!")
                        break
                    finally:
                        socket.setdefaulttimeout(old_timeout)
                except Exception as e:
                    print(f"  Failed: {e}")
                    # Remove partial download if exists
                    if os.path.exists(filepath):
                        os.remove(filepath)
                    continue
            
            if not downloaded:
                raise RuntimeError(
                    f"Could not download {f} from any source.\n"
                    f"Please download manually from:\n"
                    f"  https://ossci-datasets.s3.amazonaws.com/mnist/{f}\n"
                    f"and place in {os.path.abspath(path)}/"
                )
    
    print("MNIST data ready!")

def load_mnist(path=None):
    """
    Load MNIST data from gzipped files.
    
    Returns:
        X_train, X_test, y_train, y_test: Training and test data/labels.
        (Follows sklearn train_test_split convention)
    """
    # Resolve path relative to module root if not specified
    if path is None:
        path = os.path.join(_find_module_root(), 'data')
    
    def load_images(filename):
        with gzip.open(os.path.join(path, filename), 'rb') as f:
            data = np.frombuffer(f.read(), np.uint8, offset=16)
        return data.reshape(-1, 784).astype(np.float32) / 255.0
    
    def load_labels(filename):
        with gzip.open(os.path.join(path, filename), 'rb') as f:
            return np.frombuffer(f.read(), np.uint8, offset=8).astype(np.int32)
    
    X_train = load_images('train-images-idx3-ubyte.gz')
    y_train = load_labels('train-labels-idx1-ubyte.gz')
    X_test = load_images('t10k-images-idx3-ubyte.gz')
    y_test = load_labels('t10k-labels-idx1-ubyte.gz')
    
    return X_train, X_test, y_train, y_test

# Download and load
download_mnist()
X_train, X_test, y_train, y_test = load_mnist()

print(f"\nTraining set: {X_train.shape[0]} images")
print(f"Test set: {X_test.shape[0]} images")
print(f"Image size: {X_train.shape[1]} pixels (28x28 flattened)")
print(f"Pixel value range: [{X_train.min():.1f}, {X_train.max():.1f}]")
print(f"Label distribution: {np.bincount(y_train)}")

In [ ]:
# Visualize some samples
fig, axes = plt.subplots(2, 10, figsize=(15, 3))

for i in range(20):
    ax = axes[i // 10, i % 10]
    ax.imshow(X_train[i].reshape(28, 28), cmap='gray')
    ax.set_title(f'{y_train[i]}')
    ax.axis('off')

plt.suptitle('Sample MNIST Images', fontsize=14)
plt.tight_layout()
plt.show()

---

## Part 2: Building the Model

We'll build a simple Multi-Layer Perceptron (MLP):
- Input: 784 features (28x28 pixels)
- Hidden Layer 1: 256 neurons + ReLU
- Hidden Layer 2: 128 neurons + ReLU
- Output: 10 classes (digits 0-9)

In [ ]:
set_seed(42)

# Build model
model = Sequential(
    Linear(784, 256),
    ReLU(),
    Dropout(0.2),
    Linear(256, 128),
    ReLU(),
    Dropout(0.2),
    Linear(128, 10)
)

# Count parameters
total_params = sum(p.size for p in model.parameters())
print(f"Model Architecture:")
print(model)
print(f"\nTotal parameters: {total_params:,}")

# Setup training
loss_fn = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)

---

## Part 3: Training Loop

In [ ]:
def evaluate(model, X, y, batch_size=256):
    """Evaluate model accuracy."""
    model.eval()
    correct = 0
    total = 0
    
    for i in range(0, len(X), batch_size):
        X_batch = Tensor(X[i:i+batch_size])
        y_batch = y[i:i+batch_size]
        
        logits = model(X_batch)
        predictions = np.argmax(logits.data, axis=1)
        
        correct += np.sum(predictions == y_batch)
        total += len(y_batch)
    
    return correct / total

def train_epoch(model, X, y, loss_fn, optimizer, batch_size=64):
    """Train for one epoch."""
    model.train()
    indices = np.random.permutation(len(X))
    total_loss = 0
    n_batches = 0
    
    for i in range(0, len(X), batch_size):
        batch_idx = indices[i:i+batch_size]
        X_batch = Tensor(X[batch_idx], requires_grad=True)
        y_batch = Tensor(y[batch_idx])
        
        # Forward
        logits = model(X_batch)
        loss = loss_fn(logits, y_batch)
        
        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        n_batches += 1
    
    return total_loss / n_batches

In [ ]:
# Training configuration
NUM_EPOCHS = 10
BATCH_SIZE = 64

# Use smaller subset for faster training (optional)
# Comment these lines to use full dataset
TRAIN_SIZE = 10000
X_train_subset = X_train[:TRAIN_SIZE]
y_train_subset = y_train[:TRAIN_SIZE]

# History tracking
history = {
    'train_loss': [],
    'train_acc': [],
    'test_acc': []
}

print(f"Training on {len(X_train_subset)} samples for {NUM_EPOCHS} epochs...")
print(f"Batch size: {BATCH_SIZE}")
print("-" * 60)

start_time = time.time()

for epoch in range(NUM_EPOCHS):
    epoch_start = time.time()
    
    # Train
    train_loss = train_epoch(model, X_train_subset, y_train_subset, 
                             loss_fn, optimizer, BATCH_SIZE)
    
    # Evaluate
    train_acc = evaluate(model, X_train_subset, y_train_subset)
    test_acc = evaluate(model, X_test, y_test)
    
    # Record history
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['test_acc'].append(test_acc)
    
    epoch_time = time.time() - epoch_start
    
    print(f"Epoch {epoch+1:2d}/{NUM_EPOCHS} | "
          f"Loss: {train_loss:.4f} | "
          f"Train Acc: {train_acc:.2%} | "
          f"Test Acc: {test_acc:.2%} | "
          f"Time: {epoch_time:.1f}s")

total_time = time.time() - start_time
print("-" * 60)
print(f"Training complete! Total time: {total_time:.1f}s")
print(f"Final test accuracy: {history['test_acc'][-1]:.2%}")

---

## Part 4: Visualizing Training Progress

In [ ]:
# Plot training curves
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Loss curve
axes[0].plot(history['train_loss'], 'b-', linewidth=2)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training Loss')
axes[0].grid(True, alpha=0.3)

# Accuracy curves
axes[1].plot(history['train_acc'], 'b-', linewidth=2, label='Train')
axes[1].plot(history['test_acc'], 'r-', linewidth=2, label='Test')
axes[1].axhline(y=0.95, color='g', linestyle='--', alpha=0.5, label='95% target')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Accuracy')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

if history['test_acc'][-1] >= 0.95:
    print("Congratulations! You achieved >95% test accuracy!")
else:
    print(f"Current accuracy: {history['test_acc'][-1]:.2%}. "
          f"Try training for more epochs or adjusting hyperparameters.")

---

## Part 5: Visualizing Predictions

In [ ]:
# Show some predictions
model.eval()

# Get predictions on test set
test_logits = model(Tensor(X_test[:100]))
test_probs = test_logits.softmax().data
test_preds = np.argmax(test_probs, axis=1)

# Find correct and incorrect predictions
correct_mask = test_preds == y_test[:100]
incorrect_indices = np.where(~correct_mask)[0]

# Show some predictions
fig, axes = plt.subplots(3, 10, figsize=(15, 5))

for i in range(30):
    ax = axes[i // 10, i % 10]
    ax.imshow(X_test[i].reshape(28, 28), cmap='gray')
    
    pred = test_preds[i]
    true = y_test[i]
    conf = test_probs[i, pred] * 100
    
    color = 'green' if pred == true else 'red'
    ax.set_title(f'{pred} ({conf:.0f}%)', color=color, fontsize=9)
    ax.axis('off')

plt.suptitle('Predictions (green=correct, red=wrong)', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Show some misclassified examples
if len(incorrect_indices) > 0:
    print("Misclassified examples:")
    
    n_show = min(10, len(incorrect_indices))
    fig, axes = plt.subplots(2, n_show, figsize=(n_show * 1.5, 4))
    
    for i, idx in enumerate(incorrect_indices[:n_show]):
        # Show image
        axes[0, i].imshow(X_test[idx].reshape(28, 28), cmap='gray')
        axes[0, i].set_title(f'True: {y_test[idx]}\nPred: {test_preds[idx]}')
        axes[0, i].axis('off')
        
        # Show probability distribution
        axes[1, i].bar(range(10), test_probs[idx])
        axes[1, i].set_ylim(0, 1)
        axes[1, i].set_xticks(range(10))
        axes[1, i].set_xlabel('Class')
    
    plt.tight_layout()
    plt.show()
else:
    print("No misclassified examples in the first 100 test samples!")

---

## Part 6: Confusion Matrix

In [ ]:
# Compute confusion matrix
model.eval()

all_preds = []
for i in range(0, len(X_test), 256):
    logits = model(Tensor(X_test[i:i+256]))
    preds = np.argmax(logits.data, axis=1)
    all_preds.extend(preds)

all_preds = np.array(all_preds)

# Build confusion matrix
confusion = np.zeros((10, 10), dtype=np.int32)
for pred, true in zip(all_preds, y_test):
    confusion[true, pred] += 1

# Plot
plt.figure(figsize=(10, 8))
plt.imshow(confusion, cmap='Blues')
plt.colorbar(label='Count')

# Add numbers
for i in range(10):
    for j in range(10):
        color = 'white' if confusion[i, j] > confusion.max() / 2 else 'black'
        plt.text(j, i, str(confusion[i, j]), ha='center', va='center', color=color)

plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.xticks(range(10))
plt.yticks(range(10))
plt.tight_layout()
plt.show()

# Per-class accuracy
print("Per-class accuracy:")
for i in range(10):
    class_acc = confusion[i, i] / confusion[i].sum()
    print(f"  Digit {i}: {class_acc:.2%}")

---

## Part 7: What the Model Learned

Let's visualize the first layer weights to see what patterns the model detected.

In [ ]:
# Get first layer weights
first_layer_weights = model[0].weight.data.T  # (256, 784)

# Show first 64 filters
fig, axes = plt.subplots(8, 8, figsize=(10, 10))

for i in range(64):
    ax = axes[i // 8, i % 8]
    w = first_layer_weights[i].reshape(28, 28)
    ax.imshow(w, cmap='RdBu', vmin=-np.abs(w).max(), vmax=np.abs(w).max())
    ax.axis('off')

plt.suptitle('First Layer Filters (Red=positive, Blue=negative)', fontsize=14)
plt.tight_layout()
plt.show()

print("Each filter looks for different patterns in the image.")
print("Red areas activate for bright pixels, blue areas activate for dark pixels.")

---

## Try It Yourself: Improve the Model

Try these modifications to improve accuracy:

1. **More training**: Increase `NUM_EPOCHS` to 20 or more
2. **Larger model**: Add more neurons or layers
3. **Different optimizer**: Try SGD with momentum
4. **Learning rate tuning**: Try 0.0001 or 0.01
5. **Full dataset**: Use all 60,000 training samples

In [ ]:
# YOUR CODE HERE: Experiment with different configurations

# Example: Larger model
# model_large = Sequential(
#     Linear(784, 512),
#     ReLU(),
#     Dropout(0.3),
#     Linear(512, 256),
#     ReLU(),
#     Dropout(0.3),
#     Linear(256, 10)
# )

---

## Checkpoint

You've accomplished:
- Loaded and preprocessed the MNIST dataset
- Built an MLP classifier with your MicroGrad+ library
- Trained to >95% accuracy on handwritten digits
- Visualized training progress and model predictions
- Analyzed model behavior with confusion matrix and filter visualization

### Performance Note

MicroGrad+ is **10-100x slower than PyTorch** by design - it's built for learning, not speed.
In Domain 2, you'll use PyTorch with GPU acceleration for production-level performance.

### Memory Management

When training on the full 60,000 samples or experimenting with larger models,
you may want to clear memory between runs:

```python
import gc
gc.collect()
```

---

## Cleanup

In [ ]:
# Cleanup - release memory
from micrograd_plus.utils import cleanup_notebook
cleanup_notebook(globals())

---

## Next Steps

Continue to the final lab:
- **Lab 1.7.6**: Documentation and Benchmarks - documenting your library and comparing with PyTorch